https://www.salesforce.com/news/all-news/

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import sys,bs4,re,time,requests,pandas as pd
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
#driver for operation
from webdriver_manager.chrome import ChromeDriverManager
option = webdriver.ChromeOptions()
option.add_argument('headless')
driver = webdriver.Chrome(ChromeDriverManager().install()
                          ,options=option
                         )



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\AKSHAY SATPUTE\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [3]:
def parse_webpage_bs(search_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

In [4]:


url = f'https://www.salesforce.com/news/all-news/'

driver.get(url)

driver.refresh()

hrefs = []

range_count = 10

for x in range(range_count):
    driver.implicitly_wait(100)
    driver.find_element_by_xpath('''/html/body/main/div/div/div/div/div/span''').click()
    print(str(range_count-(x+1)) + " pages remaining..",end="\r")

links = driver.find_elements_by_xpath('''/html/body/main/div/div/div/div/a''')

for l in links:
    hrefs.append(l.get_attribute("href"))

CPU times: total: 109 ms
Wall time: 17.4 s


In [5]:


articles  = []
dates=[]
titles=[]
thumbnails=[]

for number,link in enumerate(hrefs):
    
    soup = parse_webpage_bs(link)

    art_containers = soup.findAll('section',{'class' :'the-content'})

    for art in art_containers:
        article = art.text
        articles.append(article)

    t_containers = soup.findAll('h1',{'class' :'hero__title'})

    for t in t_containers:
        title = t.text
        titles.append(title)

    d_containers = soup.findAll('p',{'class' :'byline__date'})

    for d in d_containers:
        date = d.text
        dates.append(date)
        
    thumb_containers = soup.findAll('figure',{'class' :'wp-block-image prevent-download'})

    for thumb in thumb_containers:
        thumbnail = thumb.find('img')['src']
        thumbnails.append(thumbnail)

    print(str(len(hrefs)-(number+1)) + " url remaining..",end="\r")

    
print()

0 url remaining....
CPU times: total: 6.92 s
Wall time: 1min 9s


In [6]:
#zippling the list so that it'll form the tuple
zipped = list(zip(dates,titles,articles,hrefs,thumbnails))
zipped[:3]

[('April 20, 2022',
  'Salesforce Invites Future Tech Leaders to San Francisco for New Computer Science Pre-Internship',
  '\n\nAt Salesforce, we love our Futureforce interns. Every summer, they share their energy, passion and innovation with us — questioning the old way of doing things, taking on projects that have a real impact on the business, and making our employees, company, and communities better for having worked with them.\nOur Futureforce program has helped us attract, retain, and cultivate the next generation of leaders at Salesforce, some of the most diverse and global employees at the company. We continue to invest in our interns and each year we convert up to 80% of our eligible interns to full-time employees.\nToday, we’re proud to expand the program with Futureforce Tech Launchpad, a new pre-internship program, powered by CodePath, that will provide underrepresented students in computer science access to hands-on technical training, capstone projects, and mentorship ear

In [7]:
#creating dataframe 
temp_df = pd.DataFrame(zipped,columns=['date','title','article','url','thumbnail'])
temp_df

,date,title,article,url,thumbnail
0,"April 20, 2022",Salesforce Invites Future Tech Leaders to San ...,"\n\nAt Salesforce, we love our Futureforce int...",https://www.salesforce.com/news/stories/salesf...,https://www.salesforce.com/news/wp-content/upl...
1,"April 20, 2022",Salesforce Announces New Customer 360 for Educ...,\n\nAccording to the Salesforce Connected Stud...,https://www.salesforce.com/news/stories/salesf...,https://www.salesforce.com/news/wp-content/upl...
2,"April 19, 2022",Salesforce Hits Half a Billion Dollars in All-...,\n\nSalesforce has reached over half a billion...,https://www.salesforce.com/news/stories/salesf...,https://www.salesforce.com/news/wp-content/upl...
3,"April 13, 2022",Salesforce Launches New Initiatives to Grow Su...,\n\nSalesforce and global environmental NGOs w...,https://www.salesforce.com/news/stories/salesf...,https://www.salesforce.com/news/wp-content/upl...
4,"April 12, 2022",Eurotunnel and Salesforce Team up to Power Dig...,\n\nEurotunnel today announced a partnership w...,https://www.salesforce.com/news/stories/eurotu...,https://www.salesforce.com/news/wp-content/upl...
...,...,...,...,...,...
91,"September 20, 2021",Salesforce Announces 2021 Partner Innovation A...,\n\nWe describe the Salesforce partner ecosyst...,https://www.salesforce.com/news/stories/salesf...,https://www.salesforce.com/news/wp-content/upl...
92,"September 20, 2021",Salesforce Gives $19M to Schools,\n\nGrants support five school districts acros...,https://www.salesforce.com/news/stories/salesf...,https://www.salesforce.com/news/wp-content/upl...
93,"September 20, 2021",New Study Finds Salesforce Economy Will Create...,\n\nSalesforce partner ecosystem on track to g...,https://www.salesforce.com/news/press-releases...,https://www.salesforce.com/news/wp-content/upl...
94,"September 17, 2021",New IDC Report Examines Salesforce’s ‘Gold Sta...,"\n\nAccording to a new IDC report, “Salesforce...",https://www.salesforce.com/news/stories/salesf...,https://www.salesforce.com/news/wp-content/upl...


In [8]:
#to csv
temp_df.to_csv('salesforce news.csv')

#to json
temp_df.to_json('salesforce news.json')